<a href="https://colab.research.google.com/github/Rafae1040/integrando-python-SQL/blob/main/Integra%C3%A7%C3%A3o_Python_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlalchemy as sqlA
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import relationship
from sqlalchemy import Column, func
from sqlalchemy import String
from sqlalchemy import Integer
from sqlalchemy import ForeignKey
from sqlalchemy import inspect
from sqlalchemy import create_engine
from sqlalchemy import select


Base = declarative_base()

class User(Base):
    __tablename__ = 'user_account'
    #atributos
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)

    address = relationship("Address", back_populates="user", cascade="all, delete-orphan")
    def __repr__(self):
        return f'User(id={self.id}, name={self.name}, fullname={self.fullname})'


class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email_address = Column(String(30), nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)

    user = relationship("User", back_populates="address")

    def __repr__(self):
        return f'Address(id={self.id}, email_address={self.email_address})'


print(User.__tablename__)
print(Address.__tablename__)

#conecxao com banco de dados
engine = create_engine("sqlite://")

#criando as classes como tablelas na db
Base.metadata.create_all(engine)

insp = inspect(engine)
print(insp.has_table("user_account"))
print(insp.default_schema_name)
print(insp.get_table_names())

with Session(engine) as session:
    rafael = User(
        name="Rafael",
        fullname="Rafael Carvalho",
        address=[Address(email_address="rafaelcarvalho@hotmail.com")]
    )

    elaine = User(
        name="Elaine",
        fullname="Elaine Carvalho",
        address=[Address(email_address="elainecarvalho@hotmail.com")]
    )

    stephanie = User(
        name="Stephanie",
        fullname="Stephanie Oliveira"
    )

    #enviar para db
    session.add_all([rafael, elaine, stephanie])

    session.commit()


stmt = select(User).where(User.name.in_(['rafael', 'rafael']))
print('Recuperando usuarios a partir de condição de filtragem')
for user in session.scalars(stmt):
    print(user)

stmt_address = select(Address).where(Address.user_id.in_([2]))
print('Recuperando usuarios a partir de condição de filtragem')
for address in session.scalars(stmt_address):
    print(address)

stmt_order = select(User).order_by(User.fullname.desc())
for result in session.scalars(stmt_order):
    print(result)

stmt_join = select(User.fullname, Address.email_address).join_from(Address, User)
for result in session.scalars(stmt_join):
    print(result)

connection = engine.connect()
results = connection.execute(stmt_join)
print('Executando statement a partir da connection')
for result in results:
    print(result)

stmt_count = select(func.count('*')).select_from(User)
print('Total de instancias em User ', end='')
for result in session.scalars(stmt_count):
    print(result)

user_account
address
True
main
['address', 'user_account']
Recuperando usuarios a partir de condição de filtragem
Recuperando usuarios a partir de condição de filtragem
Address(id=2, email_address=elainecarvalho@hotmail.com)
User(id=3, name=Stephanie, fullname=Stephanie Oliveira)
User(id=1, name=Rafael, fullname=Rafael Carvalho)
User(id=2, name=Elaine, fullname=Elaine Carvalho)
Rafael Carvalho
Elaine Carvalho
Executando statement a partir da connection
('Rafael Carvalho', 'rafaelcarvalho@hotmail.com')
('Elaine Carvalho', 'elainecarvalho@hotmail.com')
Total de instancias em User 3


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Table, MetaData, ForeignKey, text

engine = create_engine('sqlite:///:memory:')

metadata_obj = MetaData()
user = Table(
    'user',
    metadata_obj,
    Column('user_id', Integer, primary_key=True),
    Column('user_name', String(40), nullable=False),
    Column('email_address', String(60)),
    Column('nickname', String(50), nullable=False)
)
sql_insert = text("insert into user values(1, 'juliana', 'email@email.com', 'ju')")
engine.execute(sql_insert)

user_prefs = Table(
    'userprefs', metadata_obj,
    Column('pref_id', Integer, primary_key=True),
    Column('user_id', Integer, ForeignKey('user.user_id'), nullable=False),
    Column('pref_name', String(40), nullable=False),
    Column('pref_value', String(100)),
)

metadata_obj.create_all(engine)

metadata_db_obj = MetaData()
financial_info = Table(
    'financial_info',
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('value', String(100), nullable=False)
)

for table in metadata_obj.sorted_tables:
    print(table)

print('Executando statement sql')
sql = text('select * from user')
print(engine.execute(sql))
result = engine.execute(sql)
for num in result:
    print(num)